In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import tensorflow as tf
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [55]:
def append_acceleration_data(csv_file, label, df):
    # Read the CSV file in chunks of 250 rows
    chunk_size = 250
    columns_to_select = ['Linear Acceleration x (m/s^2)', 'Linear Acceleration y (m/s^2)', 'Linear Acceleration z (m/s^2)']

    for chunk in pd.read_csv(csv_file,usecols=columns_to_select, chunksize=chunk_size):
        if len(chunk) < chunk_size:
            break
        # Select the first 250 columns
        selected_columns = chunk.iloc[:, :250]
        
        # Reshape the selected columns into a 3-dimensional array
        acceleration_data = selected_columns.values

        
        # Create a DataFrame for the acceleration data
        acceleration_data = selected_columns.values.reshape(-1, 3, 250)
        acceleration_df = pd.DataFrame({'Acceleration': [acceleration_data]})

        
        # Create a DataFrame for the label column
        label_df = pd.DataFrame({'Label': [label] * len(acceleration_df)})
        
        # Concatenate the acceleration and label DataFrames
        result_df = pd.concat([acceleration_df, label_df], axis=1)
        
        # Append the result DataFrame to the main DataFrame
        df = pd.concat([df, result_df], ignore_index=True)
    return df

# Initialize an empty DataFrame
df = pd.DataFrame(columns=['Acceleration', 'Label'])
folder_path = '/'

# Provide the CSV file path and label value
# Define label mappings
label_mappings = {
    'Helthy': 0,
    'Suffling': 1
}
csv_files = glob.glob( '*.csv')

for csv_file in csv_files:
    # Extract the label from the file name
    file_name = csv_file.split('/')[-1]  # Extract file name from the path
    label = None
    for key in label_mappings.keys():
        if key.lower() in file_name.lower():
            label = label_mappings[key]
            break
    
    # If label is not found in the mappings, skip the file
    if label is None:
        continue
    
    # Append the acceleration data and label to the DataFrame
    df = append_acceleration_data(csv_file, label, df)

# Display the resulting DataFram

In [27]:
csv_files

['Helthy_ahsan.csv',
 'Helthy_ahsan_0.csv',
 'Helthy_mazar.csv',
 'Suffling_ahsan.csv',
 'Suffling_ahsan_0.csv',
 'Suffling_mazar.csv']

In [57]:


# Split the DataFrame into features (acceleration data) and labels
X = np.array(df['Acceleration'].tolist())
y = np.array(df['Label'].tolist())

# Preprocess the acceleration data if required
# ...

# Perform label encoding if labels are not numerical
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the feature data for CNN input
X_train = X_train.reshape(X_train.shape[0], 3, 250, 1)
X_test = X_test.reshape(X_test.shape[0], 3, 250, 1)

# Convert the labels to categorical format
num_classes = len(np.unique(y))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Define the CNN model
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 200), activation='relu', input_shape=(3, 250, 1)))
model.add(MaxPooling2D(pool_size=(1, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1)

# Evaluate the model on training data
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
print('Train accuracy:', train_accuracy)

# Evaluate the model on testing data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', test_accuracy)


Epoch 1/10
3/3 [==============================] - 1s 14ms/step - loss: 0.8167 - accuracy: 0.5529
Epoch 2/10
3/3 [==============================] - 0s 19ms/step - loss: 0.3202 - accuracy: 0.8824
Epoch 3/10
3/3 [==============================] - 0s 18ms/step - loss: 0.1492 - accuracy: 0.9529
Epoch 4/10
3/3 [==============================] - 0s 18ms/step - loss: 0.0878 - accuracy: 0.9529
Epoch 5/10
3/3 [==============================] - 0s 16ms/step - loss: 0.0657 - accuracy: 0.9647
Epoch 6/10
3/3 [==============================] - 0s 16ms/step - loss: 0.0542 - accuracy: 0.9647
Epoch 7/10
3/3 [==============================] - 0s 17ms/step - loss: 0.0460 - accuracy: 0.9882
Epoch 8/10
3/3 [==============================] - 0s 19ms/step - loss: 0.0408 - accuracy: 1.0000
Epoch 9/10
3/3 [==============================] - 0s 17ms/step - loss: 0.0360 - accuracy: 1.0000
Epoch 10/10
3/3 [==============================] - 0s 16ms/step - loss: 0.0326 - accuracy: 1.0000
Train accuracy: 1.0
Test accu

In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Split the DataFrame into features (acceleration data) and labels
X = np.array(df['Acceleration'].tolist())
y = np.array(df['Label'].tolist())

# Preprocess the acceleration data if required
# ...

# Perform label encoding if labels are not numerical
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the feature data for LSTM input
X_train = X_train.reshape(X_train.shape[0], 3, 250)
X_test = X_test.reshape(X_test.shape[0], 3, 250)

# Convert the labels to categorical format
num_classes = len(np.unique(y))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Define the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(3, 250)))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=30, verbose=1)

# Evaluate the model on training data
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
print('Train accuracy:', train_accuracy)

# Evaluate the model on testing data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', test_accuracy)


Epoch 1/30
3/3 [==============================] - 3s 9ms/step - loss: 0.7018 - accuracy: 0.5529
Epoch 2/30
3/3 [==============================] - 0s 8ms/step - loss: 0.5122 - accuracy: 0.7529
Epoch 3/30
3/3 [==============================] - 0s 9ms/step - loss: 0.4089 - accuracy: 0.9412
Epoch 4/30
3/3 [==============================] - 0s 8ms/step - loss: 0.3345 - accuracy: 0.9882
Epoch 5/30
3/3 [==============================] - 0s 7ms/step - loss: 0.2751 - accuracy: 0.9882
Epoch 6/30
3/3 [==============================] - 0s 8ms/step - loss: 0.2280 - accuracy: 0.9882
Epoch 7/30
3/3 [==============================] - 0s 8ms/step - loss: 0.1887 - accuracy: 0.9882
Epoch 8/30
3/3 [==============================] - 0s 8ms/step - loss: 0.1561 - accuracy: 0.9882
Epoch 9/30
3/3 [==============================] - 0s 9ms/step - loss: 0.1301 - accuracy: 0.9882
Epoch 10/30
3/3 [==============================] - 0s 8ms/step - loss: 0.1079 - accuracy: 1.0000
Epoch 11/30
3/3 [======================